In [144]:
import requests
import json

Set the drug that we want to find associations for

In [145]:
myDrug = "INFLIXIMAB"

Make the API call to find all the drugs

In [146]:
esURL = 'http://172.17.7.25:80/16.08_evidence-data-generic/_search'

query = json.dumps({
        "fields": ["disease.efo_info.therapeutic_area.labels", "drug.molecule_name", "target.gene_info.symbol"],
        "query": {
            "match": {
                "drug.molecule_name": myDrug
            }
        }
    })

response = requests.get(esURL, data=query)
output = response.json()

#print out the json for debug
#print json.dumps(output, indent=2)

Collect information about diseases and targets associated with our drug

In [147]:

for data in output['hits']['hits']:
    
    #print json.dumps(data, indent=2)

    for x in data['fields']['disease.efo_info.therapeutic_area.labels']:
        diseases[x] = 'true'
    for y in data['fields']['target.gene_info.symbol']:
        targets[y] = 'true'

print "DRUG: " + myDrug

print "\nDISEASES: "

for key in diseases.keys():
    print "\t" + key
    
print "\nTARGETS:"

for key in targets.keys():
    print "\t" + key

DRUG: INFLIXIMAB

DISEASES: 
	eye disease
	skeletal system disease
	immune system disease
	skin disease
	cardiovascular disease
	nervous system disease
	digestive system disease

TARGETS:
	TNF
